In [ ]:
import os
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import shutil
import pandas as pd
from source import image_id_converter as img_idc
from source import sort_img_files as sif
from source import detect_persons_yolo as dpy

In [ ]:
import sys
print(sys.prefix)

In [ ]:
os.getcwd()

## Set paths:

In [ ]:
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/project')

In [ ]:
root_path = Path(os.getcwd())


In [ ]:

# Define paths
image_dir = root_path/"../data"  # Replace with your directory containing images
output_dir_with_person = root_path/"../with_person"  # Replace with output directory for images with persons
output_dir_without_person = root_path/"../without_person"  # Replace with output directory for images without persons


In [ ]:
print(image_dir)
print(output_dir_with_person)
print(output_dir_without_person)

## Create directories for sorting the images:

In [ ]:
# Create output directories
#os.chdir(root_path/'..')
os.makedirs(output_dir_with_person, exist_ok=True)
os.makedirs(output_dir_without_person, exist_ok=True)
#os.chdir('root_path')

## Define the pretrained model:

In [ ]:
# Load the YOLOv5 model
model = YOLO("yolov8n.pt")  # Use yolov8n (nano) for faster inference


## Loop through images, sort them into the respective output folders according to person detection result and store results in list:

In [ ]:
# img_ids, with_person = sif.sort_img_files(image_dir, model, output_dir_with_person, 
#                                           output_dir_without_person, threshold=0.25)
#img_ids, with_person = sif.detect_persons_yolo(image_dir, model, output_dir_with_person, 
#                                         output_dir_without_person, threshold=0.25)

img_ids, with_person = dpy.detect_persons_yolo(image_dir, model, threshold=0.25, file_format="tif")

## Load person predictions into a dataframe: 

In [ ]:
results_person = pd.DataFrame({'image_id': img_ids, 'with_person': with_person})
results_person.head()


## Add one-hot-coded person predictions:

In [ ]:
results_person['with_person_pred']= [1 if x else 0 for x in results_person.with_person]
results_person.head()

In [ ]:
os.listdir(image_dir)

In [ ]:
image_file_path = image_dir / 'BernerOberland043.tif'
image_file_path 

## Load person label data:

The file with_without_person.csv contains labels added by (human) visual inspection. The labels thus represent the ground truth regarding to whether or not an image contains a person. The column with_person indicates whether a person or several persons are in the image, the columns recognisable indicates whether such person would be recognisable to a human familiar with the person in question based on their appearance (according to the jugdement of the author).

In [ ]:
with_without_person = pd.read_csv(image_dir/'with_without_person_mod.csv')
with_without_person


In [ ]:
img_ids = list(with_without_person.image_id)

In [ ]:
with_without_person['image_id'] = img_idc.reconvert_image_ids(img_ids)

In [ ]:
with_without_person.head()

## Rename the labels:

In [ ]:
with_without_person.rename(columns={'with_person': 'person_label', 'recognisable': 'recognisable_label'}, inplace=True)
with_without_person.head()


## Merge label data with the predictions:

In [ ]:
labels_results = with_without_person.merge(results_person, how='inner', on='image_id')
labels_results.head()

In [ ]:
labels_results.shape

## Calculate sensitivity and specificity for person predictions and get lists images with positive person predictions:

In [ ]:
positive_bools = labels_results.person_label == 1
negative_bools = labels_results.person_label == 0
positive_pred_bools = labels_results.with_person_pred == 1
negative_pred_bools = labels_results.with_person_pred == 0

positives = labels_results[positive_bools]
negatives = labels_results[negative_bools]
true_positives = labels_results[positive_bools & positive_pred_bools]
true_negatives = labels_results[negative_bools & negative_pred_bools]

false_negatives = labels_results[positive_bools & negative_pred_bools]
false_positives = labels_results[negative_bools & positive_pred_bools]

sensitivity = true_positives.shape[0] / positives.shape[0]
print('sensitivity:')
print(sensitivity)

specificity = true_negatives.shape[0] / negatives.shape[0]
print('specificity:')
print(specificity)


## Inspect false negatives:

In [ ]:
false_negatives

## Inspect false positives:

In [ ]:
false_positives

In [ ]:
print(f'True Positives: {true_positives.shape[0]}')
print(f'False Positives: {false_positives.shape[0]}')
print(f'True Negatives: {true_negatives.shape[0]}')
print(f'False Negatives: {false_negatives.shape[0]}')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate confusion matrix
cm = confusion_matrix(labels_results.recognisable_label, labels_results.with_person_pred)

number_true_positives = true_positives.shape[0]
number_false_positives = false_positives.shape[0]
number_true_negatives = true_negatives.shape[0]
number_false_negatives = false_negatives.shape[0]

sensitivity = number_true_positives / positives.shape[0]
specificity = number_true_negatives / negatives.shape[0]
precision = number_true_positives / (number_true_positives + number_false_positives)
miss_rate = number_false_negatives / positives.shape[0]
f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)

print("Confusion Matrix:")

plt.figure(figsize=(8,6))
confusion_matrix_data = [[number_true_negatives, number_false_positives], 
                          [number_false_negatives, number_true_positives]]
sns.heatmap(confusion_matrix_data, annot=True, fmt='d', 
            xticklabels=['Predicted Negative', 'Predicted Positive'], 
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

print(f'True Positives: {number_true_positives}')
print(f'False Positives: {number_false_positives}')
print(f'True Negatives: {number_true_negatives}')
print(f'False Negatives: {number_false_negatives}')
print(f'\nSensitivity (Recall): {sensitivity:.4f}')
print(f'Specificity: {specificity:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Miss Rate (False Negative Rate): {miss_rate:.4f}')
print(f'F1 Score: {f1_score:.4f}')

In [ ]:
plt.figure(figsize=(15,8))
gs = plt.GridSpec(1, 2, width_ratios=[2, 1])

plt.subplot(gs[0])
confusion_matrix_data = [[number_true_negatives, number_false_positives], 
                         [number_false_negatives, number_true_positives]]
heatmap = sns.heatmap(confusion_matrix_data, annot=True, fmt='d', 
           xticklabels=['Predicted Negative', 'Predicted Positive'], 
           yticklabels=['Actual Negative', 'Actual Positive'],
           cbar_kws={'label': 'Number of Instances'})
plt.title('Confusion Matrix')

plt.subplot(gs[1])
plt.axis('off')
metrics_text = (f'Performance Metrics:\n\n'
               f'True Positives: {number_true_positives}\n'
               f'False Positives: {number_false_positives}\n'
               f'True Negatives: {number_true_negatives}\n'
               f'False Negatives: {number_false_negatives}\n\n'
               f'Sensitivity: {sensitivity:.4f}\n'
               f'Specificity: {specificity:.4f}\n'
               f'Precision: {precision:.4f}\n'
               f'Miss Rate: {miss_rate:.4f}\n'
               f'F1 Score: {f1_score:.4f}')
plt.text(0, 0.5, metrics_text, fontsize=10, 
        verticalalignment='center')

plt.suptitle('Confusion Matrix and Performance Metrics Based on the Person Label as Ground Truth', fontsize=16)
plt.tight_layout()
plt.savefig('confusion_matrix_metrics_person.pdf')
plt.close()

## Recalculate Measures with recognisable_label as ground truth (instead of person_label):

In [ ]:
positive_bools = labels_results.recognisable_label == 1
negative_bools = labels_results.recognisable_label == 0
positive_pred_bools = labels_results.with_person_pred == 1
negative_pred_bools = labels_results.with_person_pred == 0

positives = labels_results[positive_bools]
negatives = labels_results[negative_bools]
true_positives = labels_results[positive_bools & positive_pred_bools]
true_negatives = labels_results[negative_bools & negative_pred_bools]

false_negatives = labels_results[positive_bools & negative_pred_bools]
false_positives = labels_results[negative_bools & positive_pred_bools]

sensitivity = true_positives.shape[0] / positives.shape[0]
print('sensitivity:')
print(sensitivity)

specificity = true_negatives.shape[0] / negatives.shape[0]
print('specificity:')
print(specificity)


In [ ]:
print(f'True Positives: {true_positives.shape[0]}')
print(f'False Positives: {false_positives.shape[0]}')
print(f'True Negatives: {true_negatives.shape[0]}')
print(f'False Negatives: {false_negatives.shape[0]}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate confusion matrix
cm = confusion_matrix(labels_results.recognisable_label, labels_results.with_person_pred)

number_true_positives = true_positives.shape[0]
number_false_positives = false_positives.shape[0]
number_true_negatives = true_negatives.shape[0]
number_false_negatives = false_negatives.shape[0]

sensitivity = number_true_positives / positives.shape[0]
specificity = number_true_negatives / negatives.shape[0]
precision = number_true_positives / (number_true_positives + number_false_positives)
miss_rate = number_false_negatives / positives.shape[0]
f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)

print("Confusion Matrix:")

plt.figure(figsize=(8,6))
confusion_matrix_data = [[number_true_negatives, number_false_positives], 
                          [number_false_negatives, number_true_positives]]
sns.heatmap(confusion_matrix_data, annot=True, fmt='d', 
            xticklabels=['Predicted Negative', 'Predicted Positive'], 
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

print(f'True Positives: {number_true_positives}')
print(f'False Positives: {number_false_positives}')
print(f'True Negatives: {number_true_negatives}')
print(f'False Negatives: {number_false_negatives}')
print(f'\nSensitivity (Recall): {sensitivity:.4f}')
print(f'Specificity: {specificity:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Miss Rate (False Negative Rate): {miss_rate:.4f}')
print(f'F1 Score: {f1_score:.4f}')

In [ ]:
plt.figure(figsize=(15,8))
gs = plt.GridSpec(1, 2, width_ratios=[2, 1])

plt.subplot(gs[0])
confusion_matrix_data = [[number_true_negatives, number_false_positives], 
                         [number_false_negatives, number_true_positives]]
heatmap = sns.heatmap(confusion_matrix_data, annot=True, fmt='d', 
           xticklabels=['Predicted Negative', 'Predicted Positive'], 
           yticklabels=['Actual Negative', 'Actual Positive'],
           cbar_kws={'label': 'Number of Instances'})
plt.title('Confusion Matrix')

plt.subplot(gs[1])
plt.axis('off')
metrics_text = (f'Performance Metrics:\n\n'
               f'True Positives: {number_true_positives}\n'
               f'False Positives: {number_false_positives}\n'
               f'True Negatives: {number_true_negatives}\n'
               f'False Negatives: {number_false_negatives}\n\n'
               f'Sensitivity: {sensitivity:.4f}\n'
               f'Specificity: {specificity:.4f}\n'
               f'Precision: {precision:.4f}\n'
               f'Miss Rate: {miss_rate:.4f}\n'
               f'F1 Score: {f1_score:.4f}')
plt.text(0, 0.5, metrics_text, fontsize=10, 
        verticalalignment='center')

plt.suptitle('Confusion Matrix and Performance Metrics Based on the Recognisable Label as Ground Truth', fontsize=16)
plt.tight_layout()
plt.savefig('confusion_matrix_metrics_recognisable.pdf')
plt.close()

In [ ]:
pred_positives = labels_results[positive_pred_bools]
pred_positives.head()

In [ ]:
img_ids_pos_pred = list(pred_positives.image_id)
img_ids_pos_pred

## Move files to corresponding folders based on person predictions:

In [ ]:

for image_path in Path(image_dir).glob("*.tif"):
    #print(image_path)
    path_str = str(image_path.resolve())
    #print(path_str)
    parts = path_str.split('.tif')
    img_id = parts[-2][-3:]
    #print(img_id)
    #print(type(img_id))
    parts = path_str.split('/')
    filename = parts[-1]
    #print(filename)
    
    
    if img_id in img_ids_pos_pred:
        file_path_origin = image_path
        file_path_dest = os.path.join(output_dir_with_person, filename)
        shutil.move(file_path_origin, file_path_dest)
    else:
        file_path_origin = image_path
        file_path_dest = os.path.join(output_dir_without_person, filename)
        shutil.move(file_path_origin, file_path_dest)





## Visually inspect the images in the two folders!

Visually verified all classified images, false negatives are all images with non-recognisable persons (according to my judgement).

## Check how many images have been moved to folder output_dir_with_person:

In [ ]:
files_pred_with_person = [f for f in os.listdir(output_dir_with_person) if f.endswith('.tif')]
#files_pred_with_person

In [ ]:
len(files_pred_with_person)

## Check how many images have been moved to folder output_dir_without_person:

In [ ]:
files_pred_without_person = tif_files = [f for f in os.listdir(output_dir_without_person) if f.endswith('.tif')]
#files_pred_without_person

In [ ]:
len(files_pred_without_person)

## Save labels and results:

In [ ]:
labels_results

In [ ]:
# Add image ids that will remain string type even when saved to csv and reloaded:
labels = list(labels_results.image_id)
new_labels = img_idc.complete_image_ids(labels)
labels_results['image_id_str'] = new_labels
labels_results

In [ ]:
os.getcwd()

In [ ]:
cols_to_select = ['image_id', 'person_label', 'recognisable_label', 'with_person_pred', 'image_id_str']

In [ ]:
labels_results_to_store = labels_results[cols_to_select].copy()
labels_results_to_store

In [ ]:
labels_results_to_store.rename({'with_person_pred': 'prediction_with_person'}, axis='columns',
                              inplace=True)

In [ ]:
labels_results_to_store

In [ ]:
labels_results_to_store.to_csv(image_dir/'results_people_detection.csv')